# Exercise 9 - Batch Inferencing with Pipelines (Parallel Step)

In the previous exercise, you used an Azure ML *pipeline* to automate the training and registration of a model, and prior to that you published a model as a web service for real-time *inferencing* (getting predictions from a model). Now you'll combine these two concepts to create a pipeline for *batch inferencing*. What does that mean? Well, imagine a health clinic takes patient measurements all day, saving the details for each patient in a separate file. Then overnight, the diabetes prediction model can be used to process all of the day's patient data as a batch, generating predictions that will be waiting the following morning so that the clinic can follow up with patients who are predicted to be at risk of diabetes. That's what we'll implement in this exercise.

> **Important**: This exercise assumes you have completed the previous exercise in this series - specifically, you must have:
>
> - Created an Azure ML Workspace.
> - Created an Azure ML Compute cluster.
> - Trained and registered a diabetes model.
>
> If you haven't done that, you'll need to do so before proceeding any further!

## Task 1: Connect to Your Workspace

The first thing you need to do is to connect to your workspace using the Azure ML SDK. Let's start by ensuring you still have the latest version installed.

In [ ]:
!pip install --upgrade azureml-sdk[notebooks]
import azureml.core
print("Ready to use Azure ML", azureml.core.VERSION)

Now you're ready to connect to your workspace.

> **Note**: If the authenticated session with your Azure subscription has expired since you completed the previous exercise, you'll be prompted to reauthenticate.

In [ ]:
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to work with', ws.name)

## Task 2: Generate and Upload Batch Data

Since we don't actually have a fully staffed clinic with patients from whom to get new data, we'll generate a random sample from our diabetes CSV file and use those to test the pipeline. Then we'll upload that data to a datastore in the Azure ML workspace.

> **Note**: In reality, you'd likely use an existing blob container that you've added to the workspace as a datastore rather than the default datastore that was created with your workspace, but we'll ignore that detail for now.

In [ ]:
from azureml.core import Datastore, Dataset
import pandas as pd
import os

# Load the diabetes data
diabetes = pd.read_csv('data/diabetes2.csv')
# Get a 100-item sample of the feature columns (not the diabetic label)
sample = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].sample(n=100).values

# Create a folder
batch_folder = './batch-data'
os.makedirs(batch_folder, exist_ok=True)
print("Folder created!")

# Save each sample as a separate file
print("Saving files...")
for i in range(100):
    fname = str(i+1) + '.csv'
    sample[i].tofile(os.path.join(batch_folder, fname), sep=",")
print("files saved!")

# Upload the files to the default datastore
print("Uploading files to datastore...")
default_ds = ws.get_default_datastore()
default_ds.upload(src_dir="batch-data", target_path="batch-data", overwrite=True, show_progress=True)

# Register a dataset for the input data
batch_data_set = Dataset.File.from_files(path=(default_ds, 'batch-data/'), validate=False)
batch_data_set = batch_data_set.register(workspace=ws, 
                                         name='batch-data',
                                         description='batch data',
                                         create_new_version=True)

print("Done!")

## Task 3: Create Compute

We'll need a compute context for the pipeline, so we'll use the Azure ML compute cluster you used in the previous exercises (it will be created if it doesn't already exist).

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # Create an AzureMl Compute resource (a container cluster)
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

## Task 4: Create a Pipeline for Batch Inferencing

Now we're ready to define the pipeline we'll use for batch inferencing. Our pipeline will need Python code to perform the batch inferencing, so let's create a folder where we can keep all the files used by the pipeline:

In [ ]:
import os
# Create a folder for the experiment files
experiment_name = 'batch_pipeline'
experiment_folder = './' + experiment_name
os.makedirs(experiment_folder, exist_ok=True)

print(experiment_folder)

Now we'll create a Python script to do the actual work, and save it in the pipeline folder:

In [ ]:
%%writefile $experiment_folder/batch_diabetes.py


import os
import numpy as np
from azureml.core import Model
import joblib


def init():
    # Runs when the pipeline step is initialized
    global model

    # load the model
    model_path = Model.get_model_path('diabetes_model')
    model = joblib.load(model_path)


def run(mini_batch):
    # runs for each batch
    resultList = []

    for f in mini_batch:
        data = np.genfromtxt(f, delimiter=',')
        prediction = model.predict(data.reshape(1, -1))
        resultList.append("{}: {}".format(os.path.basename(f), prediction[0]))
    return resultList

Next we'll define a run context that includes the dependencies required by the script

In [ ]:
from azureml.core import Environment
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.core.runconfig import CondaDependencies

# Add dependencies required by the model
# For scikit-learn models, you need scikit-learn
# If the model was trained using AutoML and includes pre-processing, you need the Azure ML AutoML package
cd = CondaDependencies.create(pip_packages=["scikit-learn", "azureml-sdk[automl]"])

batch_env = Environment(name="batch_environment")
batch_env.python.conda_dependencies = cd
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE
print("Configuration ready.")

We're going to use a pipeline to run the batch prediction script. To do that we'll import the libraries needed to use a parallel run step in a pipeline.

In [ ]:
!pip install azureml-pipeline-core
!pip install azureml-contrib-pipeline-steps

OK, now we're ready to define a pipeline step, which will run the Python script to load the model, use it to generate predictions from the input data, and save the results as a text file in the output folder.

In [ ]:
from azureml.contrib.pipeline.steps import ParallelRunConfig, ParallelRunStep
from azureml.pipeline.core import PipelineData

model = ws.models['diabetes_model']

output_dir = PipelineData(name="inferences", 
                          datastore=default_ds, 
                          output_path_on_compute="diabetes/results")

parallel_run_config = ParallelRunConfig(
    source_directory=experiment_folder,
    entry_script="batch_diabetes.py",
    mini_batch_size="5",
    error_threshold=10,
    output_action="append_row",
    environment=batch_env,
    compute_target=cpu_cluster,
    node_count=4)

from azureml.contrib.pipeline.steps import ParallelRunStep

parallelrun_step = ParallelRunStep(
    name="batch-score-diabetes",
    models=[model],
    parallel_run_config=parallel_run_config,
    inputs=[batch_data_set.as_named_input("diabetes_batch")],
    output=output_dir,
    arguments=[],
    allow_reuse=True
)

Now it's time to put all of the pieces together in a pipeline, and run it.

In [ ]:
from azureml.core import Experiment
from azureml.pipeline.core import Pipeline
from azureml.widgets import RunDetails

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])
pipeline_run = Experiment(ws, experiment_name).submit(pipeline)
RunDetails(pipeline_run).show()
pipeline_run.wait_for_completion(show_output=True)

When the pipeline has finished running, the resulting predictions will have been saved in the outputs of the experiment associated with the first (and only) step in the pipeline. We can retrieve it as follows:

In [ ]:
import pandas as pd
import shutil

shutil.rmtree("diabetes-results", ignore_errors=True)

prediction_run = next(pipeline_run.get_children())
prediction_output = prediction_run.get_output_data("inferences")
prediction_output.download(local_path="diabetes-results")


for root, dirs, files in os.walk("diabetes-results"):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

## Task 5: Publish the Pipeline and use its REST Interface

Now that you have a working pipeline for batch inferencing, you can publish it and use a REST endpoint to run it from an application.

In [ ]:
published_pipeline = pipeline_run.publish_pipeline(
    name="Diabetes_Parallel_Batch_Pipeline", description="Batch scoring of diabetes data", version="1.0")

published_pipeline

Note that the published pipeline has an endpoint, which you can see in the Azure portal. You can also find it as a property of the published pipeline object:

In [ ]:
rest_endpoint = published_pipeline.endpoint
print(rest_endpoint)

To use the endpoint, client applications need to make a REST call over HTTP. This request must be authenticated, so an authorization header is required. To test this out, we'll use the authorization header from your current connection to your Azure workspace, which you can get using the following code:

> **Note**: A real application would require a service principal with which to be authenticated.

In [ ]:
from azureml.core.authentication import InteractiveLoginAuthentication

interactive_auth = InteractiveLoginAuthentication()
auth_header = interactive_auth.get_authentication_header()
print(auth_header)

Now we're ready to call the REST interface. The pipeline runs asynchronously, so we'll get an identifier back, which we can use to track the pipeline experiment as it runs:

In [ ]:
import requests

rest_endpoint = published_pipeline.endpoint
response = requests.post(rest_endpoint, 
                         headers=auth_header, 
                         json={"ExperimentName": "Batch_Pipeline_via_REST"})
run_id = response.json()["Id"]
run_id

Since we have the run ID, we can use the **RunDetails** widget to view the experiment as it runs:

In [ ]:
from azureml.pipeline.core.run import PipelineRun
from azureml.widgets import RunDetails

published_pipeline_run = PipelineRun(ws.experiments["Batch_Pipeline_via_REST"], run_id)
RunDetails(published_pipeline_run).show()

As before, the results are in the output of the first pipeline step:

In [ ]:
import pandas as pd
import shutil

shutil.rmtree("diabetes-results", ignore_errors=True)

prediction_run = next(published_pipeline_run.get_children())
prediction_output = prediction_run.get_output_data("inferences")
prediction_output.download(local_path="diabetes-results")


for root, dirs, files in os.walk("diabetes-results"):
    for file in files:
        if file.endswith('parallel_run_step.txt'):
            result_file = os.path.join(root,file)

# cleanup output format
df = pd.read_csv(result_file, delimiter=":", header=None)
df.columns = ["File", "Prediction"]

# Display the first 20 results
df.head(20)

Now you have a pipeline that can be used to batch process daily patient data.

**More Information**: For more details about using pipelines for batch inferencing, see the [How to Run Batch Predictions](https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-run-batch-predictions) in the Azure Machine Learning documentation.